In [34]:
import Table_Reader as TR
import LCCall as LCC
import LCAnalyses as LCA
import LCPlot as LCP
import importlib
import matplotlib.pyplot as plt
import scipy.special as fc
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle
importlib.reload(LCC)
importlib.reload(LCA)
importlib.reload(LCP)
importlib.reload(TR)

<module 'Table_Reader' from '/media/lyhee/Secdrive/TSmain/Pip_test/trigger_codes_github/Table_Reader.py'>

In [37]:
%matplotlib tk

In [38]:
input_dir = "/mnt/Secdrive/TSmain/TSmain/" 
rgs  =['IC348','NGC1333','NGC2024','NGC2068','OMC23','OPHCORE','SERPM','SERPS'] # Region names
# Below is the RADEC coordinates of the map center [degree]
rra  =[56.075, 52.225, 85.42083, 86.55417, 83.87917, 246.77083, 277.45417, 277.50833] 
rdec =[32.08306, 31.28111, -1.8975, -0.10139, -5.01056, -24.54361, 1.25556, -2.04667]
#indices = np.where(np.sqrt(np.array(data.RAs)**2+np.array(data.DECs)**2)/60 < 18)

In [408]:
input_dir

'/mnt/Secdrive/TSmain/TSmain/'

# Main Cell

In [39]:
writing=0 # Writter Flag
query = []
importlib.reload(TR)
if writing:
    file = open(input_dir+"All_Bright_deg.tab",'w')
    file.write("#Index"+'\t'+"Region"+'\t'+"ID"+'\t'+"RA"+'\t'+"DEC"+"\t"+"Dist"+'\t'+"Type"+'\t'
               +"F_mean"+'\t'+"chi2_Null"+'\t'+"P_Null"+'\t'
               +'Slope'+'\t'+"dSlope"+'\t'+"S_dS"+'\t'+"chi2_Lin"+'\t'+'FAP_Lin'+'\t'
               +"Mean_Sin"+'\t'+"Period"+'\t'+"Amp"+'\t'+"midphase"+'\t'+"chi2_Sin"+'\t'
               +"FAP_Single"+'\t'+"FAP_Sin_mod"+'\t'+"F_amp"+'\t'
               +"Freq_min"+'\t'+"Freq_max"+'\t'+"dFreq"+'\t'+"F_max"+'\t'+"F_min"+'\t'+"STCH"
               +'\t'+"SD_obs"+'\t'+"DF"+'\t'+"Tester"+'\t'+"sd_fid"+"\n")
                #L = [j,rgs[i],data.IDs[j], data.RAs[j], data.DECs[j],Dist, Type,
                 #data.mean_pfluxes[j], chi2_Null/(num-1), P_Null,
                 #Slope*365.25, dSlope*365.25, Slope/dSlope, chi2_Lin/(num-2), FAP_Lin,
                 #modelpar[0], period/365.25, modelpar[2],modelpar[3],chi2_Sin/(num-3), 
                 #FAP_Sin_single, FAP_Sin*nfreqs,
                 #freq_min*365.25, freq_max*365.25, dfreq*365.25]


Index = 1
counter = 1
plot = 0
fluxes_lim = 0.014*10
sigma_sep = 1
hex_coord = 0
output_dir = input_dir+"LCs_test/"
chi2_80 = []
chi2_80s_n = []
chi2_80s_l = []
chi2_80s_s = []
FAPs_Lin = []
FAPs_Sin = []
FAPs_Sin_mod = []
checkers = []
checkersSig = []
whole_mean = []
fid_err_80 = []
PorD = []
F_mean_80 = []
Sigs_80 = []
Sigs = []
Sigs_check =[]
chi2_80s_plott = []
chi2_80s_check = []
cal_err= 0.02
Index2 = 0
kk=0

# Calculate the basic stat numbers of the sources
for i in range(0,8): # For 8 regions
    data = TR.ReadTable(input_dir+'Data/'+rgs[i]+'_source_data.dat',raw=0, datecut=2458880.5,recal=0)
    indices = [int(x) for x in data.Indices]
    data.JDs = np.array(data.JDs) - 2400000.5 # JD to MJD

    for j in indices:
        if data.IDs[j] == "JCMTPP_J053524.2-050932": 
            # Exclude the stochastic source with artifact (see Y.-H. Lee et al. 2021)
            continue
        Dist = np.sqrt((rra[i]-data.RAs[j])**2+(rdec[i]-data.DECs[j])**2)*60*60 #Distance from center
        if data.mean_pfluxes[j] > fluxes_lim and Dist < 1080: #Limiting samples to high signal to noise samples.
            num = len(data.pfluxes[j])
            num10 = int(np.round(num/10))
            data_sort = sorted(data.pfluxes[j])
            data_minmax = np.sum(data_sort[:num10] + data_sort[-num10:])
            F_mean_80.append((data.mean_pfluxes[j]*num-data_minmax)/(num-2*num10))
            fid_err_80.append(np.sqrt(0.014**2+(cal_err*F_mean_80[-1])**2))
            chi2_80.append(np.sum(abs(np.array(data_sort[num10:-num10])-F_mean_80[-1])**2/fid_err_80[-1]**2)/(num-2*num10))
            whole_mean.append(data.mean_pfluxes[j])
        kk += 1
#aaa=aaa
chi2_80=np.array(chi2_80)


for i in range(0,8): # For 8 regions
    print(rgs[i])
    data = TR.ReadTable(input_dir+'Data/'+rgs[i]+'_source_data.dat',raw=0,datecut=2458880.5,recal=0)
    indices = [int(x) for x in data.Indices]
    data.JDs = np.array(data.JDs) - 2400000.5
    
    
    for j in indices: # For every sources,
        if data.IDs[j] == "JCMTPP_J053524.2-050932":
            # Excluding the JCMTPP_J053524.2-050932 (unrealistic dip)
            # (see Y.-H. Lee et al. 2021)
            continue
        Index2 += 1
        Dist = np.sqrt((rra[i]-data.RAs[j])**2+(rdec[i]-data.DECs[j])**2)*60*60
        RAout = data.RAs[j]
        DECout = data.DECs[j]
        
        # Convert to sexagesimal coordinates. ex) 15:00:36 -01:39:52
        if hex_coord: 
            ratemp = Angle(data.RAs[j],u.degree)
            RAout = '{:02d}'.format(int(ratemp.hms[0]))+'h'+'{:02d}'.format(int(ratemp.hms[1]))+'m'+ '{:04.1f}'.format(ratemp.hms[2]) +'s'
            dectemp = Angle(data.DECs[j],u.degree)
            DECout= '{:02d}'.format(int(dectemp.dms[0]))+'d'+'{:02d}'.format(abs(int(dectemp.dms[1])))+'m'+ '{:04.1f}'.format(abs(dectemp.dms[2])) +'s'
            if dectemp.degree<0:
                DECout = "-"+ DECout
            else :
                DECout = "+"+ DECout
                
        if data.mean_pfluxes[j] > fluxes_lim and Dist < 1080:
            # Setting the type of the source
            # X == Starless : no matching protostar or disk
            # P == Protostar (class 0/I)
            # D == Disk (class II)
            Type = "X" 
            if np.min([data.proto_dists[j], data.disk_dists[j]]) < 10:
                if data.proto_dists[j] < data.disk_dists[j]: Type = "P" 
                else: Type = "D"
            PorD.append(Type)
            
            # Calculate fiducial error (see the paper)
            fid_err = np.sqrt(0.014**2+(cal_err*data.mean_pfluxes[j])**2)
            F_max = np.max(data.pfluxes[j])
            F_min = np.min(data.pfluxes[j])
            num = len(data.pfluxes[j])
            
            # Under Null Hypothesis
            chi2_Null = np.sum(abs(np.array(data.pfluxes[j])-data.mean_pfluxes[j])**2/fid_err**2)
            P_Null = fc.gammainc((num-1)/2, chi2_Null/2)
            
            # Linear Fitting
            sol_para,sol_err=LCA.LCLinearfit(data.JDs,data.pfluxes[j],[fid_err for x in data.pfluxes[j]])
            Slope = sol_para[0]
            dSlope = sol_err[0]

            linfit = np.array(data.JDs)*sol_para[0] + sol_para[1]
            chi2_Lin = np.sum(abs(np.array(data.pfluxes[j])-linfit)**2/fid_err**2)
            P_Lin = fc.gammainc((num-2)/2, chi2_Lin/2)
            Pow_Lin = (chi2_Null-chi2_Lin)/chi2_Null
            FAP_Lin = (1-Pow_Lin)**(num/2)
            data_Lin = np.array(data.pfluxes[j])-linfit
            FAPs_Lin.append(FAP_Lin)
            
            # Periodogram (Sinusoidal fitting)
            ls_source = LCA.LCLombscargle(data.JDs,data.pfluxes[j],data.sd_fids[j])
            FAP_Sin = ls_source.FAPs[np.where(ls_source.powers == np.max(ls_source.powers))][0]
            JD_center = (min(data.JDs) + max(data.JDs)) / 2
            Power_Sin = np.max(ls_source.powers)
            
            FAP_Sin_single = ls_source.FAP_single
            FAPs_Sin.append(FAP_Sin_single)
            modelpar = ls_source.modelpara() # modelpar = [mean, freq, amp, iphase]
            sinfit = ls_source.mulsin(data.JDs, modelpar)
            chi2_Sin = np.sum(abs(np.array(data.pfluxes[j])-sinfit)**2/fid_err**2)
            P_Sin = fc.gammainc((num-3)/2, chi2_Sin/2)
            data_Sin = np.array(data.pfluxes[j])-sinfit
            period = np.round(1/modelpar[1])
            freq_min = np.min(ls_source.freqs)
            freq_max = np.max(ls_source.freqs)
            dfreq = ls_source.freqs[1]-ls_source.freqs[0]
            num_freqs=int(modelpar[1]/dfreq)+1 # number of frequencies lower than the best-fit frequency
            FAPs_Sin_mod.append(FAP_Sin_single*num_freqs)
            
            #%%%%%%%%%%%%% Caution %%%%%%%%%%%%%%%%%%
            # Do Not use iphase in any analysis until it fixed by someone.
            # The model reproduction with this iphase works perfect but the real outcome of the value itself is vague
            # maybe the phase at dates = 0
            iphase = (modelpar[3]/np.pi*2)#+(JD_center%period/period) 
            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            #print([JD_center,period,JD_center%period/period])
            
            obs_err = np.std(data.pfluxes[j])
            
            # Measure the stats. values after cutting 20% from top and bottom, after subtracting each models.
            F_mean_80_n, fid_err_80_n, chi2_80_n, Sig_max_n, maxe_n =TR.Cutto80(np.array(data.pfluxes[j])-F_mean_80[Index-1],mean=F_mean_80[Index-1])
            F_mean_80_l, fid_err_80_l, chi2_80_l, Sig_max_l, maxe_l =TR.Cutto80(data_Lin, mean=F_mean_80[Index-1])
            F_mean_80_s, fid_err_80_s, chi2_80_s, Sig_max_s, maxe_s =TR.Cutto80(data_Sin, mean=F_mean_80[Index-1])
            chi2_80s_n.append(chi2_80_n)
            chi2_80s_l.append(chi2_80_l)
            chi2_80s_s.append(chi2_80_s)
            
            
            checkerSig = 0 # Is there any > 5 sigma event?
            sig_out = 0 # Capture the significance(stochasticity) of the > 5 sigma event
            checker1 = 0 # Does the source show linear trend? 
            checker2 = 0 # Does the source show sinusoidal trend? 
            checker3 = np.array([0,0,0]) # > 5 sigma event after subtracting each model? ([null, linear, sinusoid])
            
            num = len(data.pfluxes[j])
            num10 = int(np.round(num/10))
            
            
            chi2_80s_check.append(chi2_Null)
            if FAP_Lin < 1e-3:
                checker1 += 1
            if FAP_Sin_single*num_freqs < 1e-3:
                checker2 += 1
                chi2_80s_plott.append(chi2_80_s)
                #print(chi2_80_s)
            else:
                chi2_80s_plott.append(chi2_80_n)
            if abs(Sig_max_n) > 5: checker3[0] = 1
            if abs(Sig_max_l) > 5: checker3[1] = 1
            if abs(Sig_max_s) > 5: checker3[2] = 1
            checkers.append([checker1, checker2, checker3])
            
            
            obs_err_sub = np.std(data.pfluxes[j])
            delF = F_max - F_min
            Tester = np.median(data.pfluxes[j]) - F_min
            Tester = F_max - np.median(data.pfluxes[j])
            if sigma_sep:
                if checker2:
                    obs_err_sub = np.std(data_Sin)
                    for k in range(0,num):
                        # Sig == Stochasticity
                        Sig = (data_Sin[k])/fid_err
                        Sig_80 = (data_Sin[k])/fid_err_80_s#/np.max([np.sqrt(chi2_80_s),1]) # to use \sigma_obs
                        Sigs.append(Sig)
                        #Sig_80 = (data.pfluxes[j][k]-F_mean_80)/fid_err_80
                        Sigs_80.append(Sig_80)
                        if abs(Sig_80) > 5:
                            checkerSig = 1
                            print([Index, j,k, data.IDs[j], F_mean_80_s, Sig_80,"sin",fid_err*Sig,fid_err*Sig/F_mean_80[j]])
                            sig_out = Sig_80
                            
                            
                elif checker1:
                    obs_err_sub = np.std(data_Lin)
                    for k in range(0,num):
                        Sig = (data_Lin[k])/fid_err
                        Sig_80 = (data_Lin[k])/fid_err_80_l#/np.max([np.sqrt(chi2_80_l),1]) # to use \sigma_obs
                        Sigs.append(Sig)
                        #Sig_80 = (data.pfluxes[j][k]-F_mean_80)/fid_err_80
                        Sigs_80.append(Sig_80)
                        
                        if abs(Sig_80) > 5:
                            checkerSig = 1
                            print([Index, j,k, data.IDs[j], Sig_80,"lin",fid_err*Sig,fid_err*Sig/F_mean_80[j]])
                            sig_out = Sig_80
                            
                else:
                    for k in range(0,num):
                        Sig = (data.pfluxes[j][k]-data.mean_pfluxes[j])/fid_err
                        Sig_80 = (data.pfluxes[j][k]-F_mean_80[Index-1])/fid_err_80[Index-1]#/np.sqrt(np.max([(chi2_80_n),1])) # to use \sigma_obs
                        Sigs.append(Sig)
                        Sigs_check.append(Sig_80) # Stochasticities without variables
                        #Sig_80 = (data.pfluxes[j][k]-F_mean_80)/fid_err_80
                        Sigs_80.append(Sig_80)
                        if abs(Sig_80) > 5:
                            checkerSig = 1
                            print([Index,j,k, data.IDs[j],F_mean_80_n,data.mean_pfluxes[j], Sig_80,"null",fid_err*Sig,fid_err*Sig/F_mean_80[j],Dist])
                            sig_out = Sig_80

            checkersSig.append(checkerSig)
            
            # If you want to write or plot the variables only, use here
            #if any(sum([checker1,checker2,checker3])):
            if sum([checker1,checker2,checkerSig]):
                plot=0
                writing=0
            
            # %%%%%%%%%%%%%%%%%%% Plotter below %%%%%%%%%%%%%%%%%
            if plot:
                plt.close()
                fig, ax = plt.subplots(2,2,figsize=[12,9])
                fig.suptitle(rgs[i]+" "+ str(indices[j]) + "_" +data.IDs[j])
                xlimits = [np.min(data.JDs)-100,np.max(data.JDs)+100]
                
                # Light Curve plot
                ax[0][0].errorbar(data.JDs,data.pfluxes[j],yerr = fid_err,color='C0',fmt='o')
                ax[0][0].set_xlim(xlimits)
                ax[0][0].plot(xlimits,[data.mean_pfluxes[j]+obs_err,data.mean_pfluxes[j]+obs_err],color='black',ls='--')
                ax[0][0].plot(xlimits,[data.mean_pfluxes[j]-obs_err,data.mean_pfluxes[j]-obs_err],color='black',ls='--')
                ax[0][0].plot(xlimits,[data.mean_pfluxes[j]+fid_err,data.mean_pfluxes[j]+fid_err],color='blue',ls=':')
                ax[0][0].plot(xlimits,[data.mean_pfluxes[j]-fid_err,data.mean_pfluxes[j]-fid_err],color='blue',ls=':')
                ax[0][0].set_xlabel("JD - 2400000.5")
                ax[0][0].set_ylabel("Peak flux [Jy/beam]")
                # Linear best-fit model
                if checker1:
                    xplot = np.linspace(xlimits[0],xlimits[1],20)
                    linfit_plot = xplot*sol_para[0] + sol_para[1]
                    ax[0][0].plot(xplot,linfit_plot,color='C1',ls='--')
                # Sinusoidal best-fit model
                if checker2:
                    xplot = np.linspace(xlimits[0],xlimits[1],1000)
                    sinfit_plot = ls_source.mulsin(xplot, modelpar)
                    ax[0][0].plot(xplot,sinfit_plot,color='C1')
                    ax[0][0].plot([JD_center,JD_center],[np.min(data.pfluxes[j]),np.max(data.pfluxes[j])],alpha=0.3,color='grey')
                    #data.pfluxes[j] = np.array(data.pfluxes[j])-sinfit
                    #ax[1][0].errorbar(data.JDs,data.pfluxes[j],yerr=fid_err,color='C0',fmt='o')
                    #print(str(counter)+" STCH.var found!")
                    #counter +=1
                data.pfluxes[j] = np.array(data.pfluxes[j]) - F_mean_80[Index-1]
            
            if plot:
                # Plot the model-subtracted light curve
                ax[1][0].set_xlim(xlimits)
                if checker2:
                    ax[1][0].errorbar(data.JDs,data_Sin,yerr=fid_err,color='C0',fmt='o')
                else:
                    ax[1][0].errorbar(data.JDs,data.pfluxes[j],yerr=fid_err,color='C0',fmt='o')
                ax[1][0].plot(xlimits,[np.std(data.pfluxes[j]),np.std(data.pfluxes[j])],color='black',ls='--')
                ax[1][0].plot(xlimits,[-np.std(data.pfluxes[j]),-np.std(data.pfluxes[j])],color='black',ls='--')
                ax[1][0].plot(xlimits,[fid_err,fid_err],color='blue',ls=':')
                ax[1][0].plot(xlimits,[-fid_err,-fid_err],color='blue',ls=':')
                ax[1][0].set_xlabel("JD - 2400000.5")
                ax[1][0].set_ylabel(r"$\Delta$Peak flux [Jy/beam]")
                
                oversig = np.where(abs(data.pfluxes[j])/fid_err_80[Index-1] > 5*np.max([np.sqrt(chi2_80[Index-1]),1]))
                #if there is any outlier (>5 sigma), note it with red circle
                # YHLee : Limited to "checker2" case because we wanted to note HOPS 373 in the last time.
                if checker2 and plot:
                    ax[1][0].scatter(data.JDs[oversig],data_Sin[oversig],s=100,edgecolors='red',facecolors='none',lw=1.5)
                elif plot:
                    ax[1][0].scatter(data.JDs[oversig],data.pfluxes[j][oversig],s=100,edgecolors='red',facecolors='none',lw=1.5)
                
                # Text color depends on whether the variability exist.
                color1 = "black"
                color2 = 'black'
                color3 = ["black","black","black"]
                fs = 13
                if checker1: color1 = "red"
                if checker2: color2 = 'red'
                if checker3[0]: color3[0] ="red"
                if checker3[1]: color3[1] ="red"
                if checker3[2]: color3[2] ="red"
                
                # Write the variability parameters on the plot
                num = len(data.pfluxes[j])
                note = np.array([0.5,0.87])
                add = np.array([0,0.025])
                ax[0][1].axis('off')
                ax[0][1].annotate(r'# of epochs = {:4d} Type:{}  D_Proto={:5.2f}"  D_Disk={:5.2f}"'.format(
                                num,Type,data.proto_dists[j],data.disk_dists[j]),
                                  xy=note,xycoords='figure fraction',fontsize=fs)
                ax[0][1].annotate(r"F_mean(80$\%$)={:5.2f}({:5.2f}) SD_fid(80$\%$)={:5.2e}({:5.2e})".format(
                                data.mean_pfluxes[j],F_mean_80[Index-1],fid_err,fid_err_80[Index-1]),
                                  xy=note-add,xycoords='figure fraction',fontsize=fs)
                ax[0][1].annotate(r"               $\chi$       $\chi_{80}$     $\sigma 80_{Max}$  epoch",
                                  xy=note-add*3, xycoords='figure fraction', fontsize =fs)
                ax[0][1].annotate("None | {:5.2f} | {:5.2f}  | {:5.2f}  | {:5d}".format(
                                np.sqrt(chi2_Null/(num-1)),np.sqrt(chi2_80_n),Sig_max_n,maxe_n+1),
                                  xy=note-add*4,xycoords='figure fraction',fontsize=fs,color=color3[0])
                ax[0][1].annotate("Lin     | {:5.2f} | {:5.2f}  | {:5.2f}  | {:5d}".format(
                                np.sqrt(chi2_Lin/(num-2)),np.sqrt(chi2_80_l),Sig_max_l,maxe_l+1),
                                  xy=note-5*add,xycoords='figure fraction',fontsize=fs,color=color3[1])
                ax[0][1].annotate("Sin     | {:5.2f} | {:5.2f}  | {:5.2f}  | {:5d}".format(
                                np.sqrt(chi2_Sin/(num-3)),np.sqrt(chi2_80_s),Sig_max_s,maxe_s+1),
                                  xy=note-6*add, xycoords='figure fraction',fontsize=fs,color=color3[2])
                ax[0][1].annotate(r"FAP_Lin) : {:5.2e}".format(FAP_Lin),
                                  xy=note-8*add, xycoords='figure fraction',fontsize=fs,color=color1)
                ax[0][1].annotate("Slope(/mean) : {:5.2e} ({:5.2e})".format(sol_para[0]*365.25, sol_para[0]*365.25/whole_mean[Index-1]),
                                  xy=note-9*add,color=color1, xycoords='figure fraction',fontsize=fs)
                
                if FAP_Sin_single < 1e-3 and FAP_Sin_single*num_freqs > 1e-3:
                    ax[0][1].annotate("FAP_Sin : {:5.2e}".format(FAP_Sin_single),
                                      xy=note-11*add,color='orange', xycoords='figure fraction',fontsize=fs)
                else:
                    ax[0][1].annotate("FAP_Sin : {:5.2e}".format(FAP_Sin_single),
                                  xy=note-11*add,color=color2, xycoords='figure fraction',fontsize=fs)
                ax[0][1].annotate("FAP_Sin*# : {:5.2e}".format(FAP_Sin_single*num_freqs),
                                  xy=note-12*add,color=color2, xycoords='figure fraction',fontsize=fs)
                ax[0][1].annotate(r"Period   A    A/Mean$_{sine}$  $\Psi_{Mid}$ ",
                                  xy=note-13*add, xycoords='figure fraction',fontsize=fs)
                
                if modelpar[3] < 0:
                    modelpar[3] += 2*np.pi
                ax[0][1].annotate("{:5.2f} | {:5.2f} | {:5.2f}   |   {:5.2f}".format(
                                period/365.25,modelpar[2],modelpar[2]/modelpar[0],iphase),
                                  xy=note-14*add, xycoords='figure fraction',color = color2,fontsize=fs)

                # Check position in the scatter plot of every sources
                # The first loop before main loop is required for this plot
                ax[1][1].scatter(whole_mean, np.sqrt(chi2_80), color="C0",s=20)
                ax[1][1].errorbar(whole_mean[Index-1], np.sqrt(chi2_80[Index-1]),
                                  yerr=np.sqrt(chi2_80_n)-np.sqrt(chi2_80_s),uplims=True,
                                 ms = 12,color="Orange",fmt='o')
                ax[1][1].set_xlabel("Mean Flux [Jy/beam]")
                ax[1][1].set_ylabel(r"$\chi$ (80$\%$)")
                ax[1][1].set_xlim(0.1,15)
                ax[1][1].set_ylim(0.3,4)
                ax[1][1].set_xscale('log')
                ax[1][1].set_yscale('log')
                #ax[1][1].set_yticklabels([0.5,1,4])

                plt.savefig(output_dir+rgs[i]+" "+ str(indices[j]) + "_" +data.IDs[j]+".jpg")
            #aaa=aaa
            
            F_amp = max(abs(data.pfluxes[j]),key=abs)/F_mean_80_n
            Abs_amp = max(abs(data.pfluxes[j]),key=abs)
            if checker2:
                F_amp = 2*modelpar[2]/(modelpar[0]-modelpar[2])
                if F_amp < 0:
                    F_amp = max(abs(data.pfluxes[j]),key=abs)/F_mean_80_n
            
            L = [j,rgs[i],data.IDs[j], RAout, DECout ,Dist, Type,
                 data.mean_pfluxes[j], chi2_Null/(num-1), P_Null,
                 Slope*365.25, dSlope*365.25, Slope/dSlope, chi2_Lin/(num-2), FAP_Lin,
                 modelpar[0], period/365.25, modelpar[2],iphase,chi2_Sin/(num-3), 
                 FAP_Sin_single, FAP_Sin_single*num_freqs, F_amp,freq_min*365.25, 
                 freq_max*365.25, dfreq*365.25, F_max, F_min, sig_out, 
                 obs_err_sub,delF,Tester,data.sd_fids[j]]
            
            L = [str(x)+'\t' for x in L[:-1]]+[str(L[-1])]
            if writing:
                #print(L)
                file.writelines(L)
                file.write('\n')
            Index += 1
            
if writing:
    file.close()
#query = set(query)

2458880.5
2458872.8455555993
IC348
[58125.29625574872, 383.0, 0.7631756024770731]
[58125.29625574872, 2134.0, 0.23772083212217385]
[58125.29625574872, 364.0, 0.6848798234854917]
[58125.29625574872, 56.0, 0.951718852655696]
[58125.29625574872, 24.0, 0.8873439895299574]
[58125.29625574872, 18.0, 0.1831253193732765]
[58125.29625574872, 17.0, 0.13507386757170453]
[58125.29625574872, 31.0, 0.00955663705545087]
[58125.29625574872, 20.0, 0.2648127874359488]
NGC1333
2458880.5
2458872.8455555993
[58125.275515049696, 1660.0, 0.015226213885359018]
[58125.275515049696, 20.0, 0.26377575248479845]
[58125.275515049696, 14941.0, 0.8903202941603438]
[58125.275515049696, 51.0, 0.711284608817568]
[58125.275515049696, 14941.0, 0.8903202941603438]
[58125.275515049696, 26.0, 0.5875198096036911]
[58125.275515049696, 25.0, 0.011020601987838744]
[58125.275515049696, 164.0, 0.4224116771322925]
[58125.275515049696, 2988.0, 0.4529034521585328]
[58125.275515049696, 18.0, 0.1819730583164427]
[58125.275515049696, 29

In [122]:
plt.close()
plt.hist(chi2_80s_s,alpha=0.2)
plt.hist(chi2_80)
plt.yscale('log')

In [218]:
importlib.reload(TR)
data = TR.ReadTable(input_dir+'Data/Recal/SERPM_source_data.dat',raw=1,recal=1)
#data = TR.ReadTable(input_dir+'Data/SERPM_source_data.dat',recal=0)

plt.close()
plt.scatter(data.JDs,data.pfluxes[0])

In [128]:
importlib.reload(LCA)
LCA.LCLinearfit(data.JDs,data.pfluxes[2],[fid_err for x in data.JDs])

case 1


(array([ 4.21551436e-05, -1.02451887e+02]),
 array([1.09865833e-05, 2.70068601e+01]))

In [375]:
Sin_mod_d

(array([  0,   1,   9,  11,  17,  77,  80,  86,  88, 102, 148, 200, 232,
        233, 234, 235, 248, 278]),)

In [12]:
%matplotlib tk

# below block need to be run

In [ ]:
table = TR.All_Table(input_dir+'All_Bright_deg.tab')

# Define the indice of protostars as "Ps"
Ps = np.where(np.array(table.Type) == "P")
Ds = np.where(np.array(table.Type) == "D")
Sin_mod_d = np.where([x[1] for x in checkers]) # Indices of the sources with sinusoidal trend
STCH_d = np.where([sum(x[2]) for x in checkers]) # Indices of the sources with stochastic event
STCH_d = np.where(np.array(checkersSig))[0][0]

varind = []
for i in range(0,len(table.FAP_Lin)):
    if table.FAP_Lin[i] < 1e-3 or table.modFAP_Sin[i] < 1e-3:
        
        varind.append(i)

In [40]:
plt.close()
fig, ax = plt.subplots()
Ps = np.where(np.array(PorD) == "P")
ax.scatter(F_mean_80, chi2_80,marker='o',color='grey', alpha=0.7,s=20)
ax.scatter(np.array(F_mean_80)[Ps], np.array(chi2_80)[Ps],color="Red",marker='o',s=20,label = "Protostar")

Sin_mod_d = np.where([x[1] for x in checkers])
STCH_d = np.where([sum(x[2]) for x in checkers])
STCH_d = np.where(np.array(checkersSig))[0][0]

ax.scatter(np.array(F_mean_80)[Sin_mod_d], np.array(chi2_80)[Sin_mod_d],
           marker='o',edgecolor="black",facecolor='none',linewidth=1.5,s=80, label=r"FAP$_{sin}$ < 0.001")
ax.scatter(np.array(F_mean_80)[STCH_d], np.array(chi2_80)[STCH_d],
           marker='+',color="black",s=160,linewidth=0.8, label=r"$\sigma_{max}$ > 5")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(0.13,15)
ax.set_ylim(0.08,14)
ax.set_xlabel(r"F$_{mean}$ [Jy/beam]",fontsize=12)
ax.set_ylabel(r"$\chi^{2}_{80\%}$/(N$_{80\%}$)",fontsize=12)
ax.legend(loc='best',fontsize=9)

In [42]:
print(np.array(F_mean_80)[Sin_mod_d])
#EC 53 : -4
#HOPS 373: 7
#HOPS 358: 5
#VLA 3: 3
names = ["EC 53", "HOPS 373", "HOPS 358", "VLA 3"]
exs = [-4,7,5,3]
#Sin_mod_d[exs[ind]]

[1.38531688 1.2106004  9.1605209  3.04387078 0.48085847 1.2935783
 0.52417557 1.24732341 0.99460815 0.24893432 0.54222203 3.89046196
 7.08732149 2.10466088 1.17439312 0.84255033 4.84555917 0.63441788]


# 2panel plot of Fmean vs Chi-square

In [15]:
plt.close()
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, LogLocator)
# the random data



fig, axes = plt.subplots(1,2,figsize=(12, 8))

# the scatter plot:
xticks = [0.14,0.2,0.3,0.4, 0.5,0.6,0.7,0.8,0.9, 1,2,3,4, 5, 6,7,8,9, 10]
yticks = [0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8,0.9, 1,2,3,4, 5, 6,7,8,9, 10]
xticklabels = [0.14,' ',' ',' ', 0.5,' ',' ',' ',' ', 1,' ',' ',' ', 5, ' ',' ',' ',' ', 10]
yticklabels = [0.1,' ',' ',' ', 0.5,' ',' ',' ',' ', 1,' ',' ',' ', 5, ' ',' ',' ',' ', 10]
#yticks = [0.1,0.5, 1, 5, 10]
#yticklabels = [0.14,0.5,1,5,10]
i = 0
for ax in axes:
    x = np.log10(F_mean_80)

    y = np.log10(chi2_80)
    if i:
        y = np.log10(chi2_80s_plott)
    ax.scatter(x, y,marker='o',color='grey', alpha=0.7,s=20)
    ax.scatter(np.array(x)[Ps], np.array(y)[Ps],color="Red",marker='o',s=20,label = "Protostar")
    ax.scatter(np.array(x)[Ds], np.array(y)[Ds],color="Blue",marker='o',s=20,label = "Disk")
    
    sec_label = r"FAP$_{\rm Mod}$ < 0.001"
    #sec_label = r"Secular variales"
    ax.scatter(np.array(x)[Sin_mod_d], np.array(y)[Sin_mod_d],
               marker='o',edgecolor="black",facecolor='none',linewidth=1.5,s=80, label=sec_label)
    
    for ind in range(0,4):
        ax.annotate(names[ind], xy=[np.array(x)[Sin_mod_d[0][exs[ind]]]+np.log10(0.85)
            ,np.array(y)[Sin_mod_d[0][exs[ind]]]+np.log10(1.115)], 
                    xycoords="data", fontsize=12)
    
    stch_label = r"|$\sigma_{max}$| > 5"
    #stch_label = r"Stochastic variable"
    ax.scatter(np.array(x)[STCH_d], np.array(y)[STCH_d],
               marker='+',color="black",s=160,linewidth=0.8, label=stch_label)

    ax.set_xlim(np.log10(0.13),np.log10(15))
    ax.set_ylim(np.log10(0.08),np.log10(18))
    ax.set_xlabel(r"$\bar{F}_{\rm 80\%}$ [Jy/beam]",fontsize=14)
    ax.set_ylabel(r"$\chi^{2}_{80\%}$/(N$_{80\%}$)",fontsize=14)
    ax.plot([np.log10(0.13),np.log10(15)],[np.log10(0.45*4),np.log10(0.45*4)],alpha=0.7,ls='--',color='red')
    ax.plot([np.log10(0.13),np.log10(15)],[np.log10(0.45),np.log10(0.45)],alpha=1,ls='--',color='black')

    ax.set_xticks([np.log10(x) for x in xticks])
    ax.set_yticks([np.log10(x) for x in yticks])
    ax.set_xticklabels([str(x) for x in xticklabels])
    ax.set_yticklabels([str(x) for x in yticklabels])
    ax.label_outer()
    ax.tick_params(labelsize=13)
#ax.xaxis.set_minor_locator(LogLocator(base=5.0,
#                                      subs=(0.1,0.2,0.3,0.4),numticks=4))

#ax.tick_params(which='minor', length=2, color='r')
# Set aspect of the main axes.

    ax.set_aspect(1)
    
    i +=1

axes[0].legend(loc='upper left', fontsize=12)
axes[1].annotate("After secular trends subtracted",xycoords="figure fraction",xy=[0.68,0.8],fontsize=14)
# create new axes on the right and on the top of the current axes

divider = make_axes_locatable(ax)
## below height and pad are in inches

#ax_histx = divider.append_axes("top", 1.0, pad=0.1, sharex=ax)
#ax_histy = divider.append_axes("right", 1.0, pad=0.1, sharey=ax)

## make some labels invisible

#ax_histx.xaxis.set_tick_params(labelbottom=False)
#ax_histx.set_yticks([1,10,100])
##ax_histx.xaxis.set_xscale("log")
#ax_histy.yaxis.set_tick_params(labelleft=False)
#ax_histy.set_xticks([1,10,100])
##ax_histy.yaxis.set_yscale("log")

## now determine nice limits by hand:
##binwidth = 0.05
binwidth = 0.3
xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
lim = (int(xymax/binwidth) + 1)*binwidth

bins = np.arange(-lim, lim + binwidth, binwidth)
#ax_histx.hist(np.array(x), bins=bins,color='grey')
#ax_histx.hist(np.array(x)[Ps], bins=bins,color='red')
#ax_histx.hist(np.array(x)[Ds], bins=bins,color='blue')
#ax_histx.hist(np.array(x)[Sin_mod_d], bins=bins,histtype='step',color='black')
#ax_histy.hist(y, bins=bins,orientation='horizontal',color='grey')
#ax_histy.hist(np.array(y)[Ps], bins=bins,orientation='horizontal',color='red')
#ax_histy.hist(np.array(y)[Ds], bins=bins,orientation='horizontal',color='blue')
#ax_histy.hist(np.array(y)[Sin_mod_d], bins=bins,orientation='horizontal',histtype='step',color='black')

# the xaxis of ax_histx and yaxis of ax_histy are shared with ax,
# thus there is no need to manually adjust the xlim and ylim of these
# axis.
##ax_histx.set_xscale("log")
##ax_histy.set_yscale("log")

#ax_histx.set_yscale("log")
#ax_histx.set_ylim([0.8, 200])
#ax_histy.set_xscale("log")
#ax_histy.set_xlim([0.8, 200])

##ax_histx.set_yticks([0, 30, 60])
##ax_histy.set_xticks([0, 30, 60])

fig.tight_layout()
fig.subplots_adjust(wspace=0.03, hspace=0)
plt.show()


In [428]:
plt.savefig(output_dir+"Hist_Fmean_80.pdf")
print(output_dir+"Hist_Fmean_80.pdf")

/mnt/Secdrive/TSmain/TSmain/LCs_test/Hist_Fmean_80.pdf


In [399]:
input_dir

'/mnt/Secdrive/TSmain/TSmain/'

In [430]:
output_dir = input_dir+'PaperFigs/'
print(output_dir)

/mnt/Secdrive/TSmain/TSmain/PaperFigs/


In [65]:
from matplotlib.ticker import FuncFormatter
plt.close()
fig, axis = plt.subplots()
bins = np.linspace(np.log10(0.14),np.log10(10),7)
logF_mean_80 = np.log10(np.array(F_mean_80))
axis.hist(logF_mean_80,bins=bins,color='grey')
axis.hist(logF_mean_80[Ps],bins=bins,color='red',label='Protostar')
axis.hist(logF_mean_80[Ds],bins=bins,color='blue',label="Disk")
axis.hist(logF_mean_80[Sin_mod_d],bins=bins,histtype='step',color='black',label=r"FAP$_{\rm Mod}$ < 0.001",linewidth=2,hatch='//')
#xticks = [0.14,0.2,0.3,0.4, 0.5,0.6,0.7,0.8,0.9, 1,2,3,4, 5, 6,7,8,9, 10]
#xticks = [np.log10(x) for x in [0.14, 0.5,1,2,5,10]]
xticks = [np.log10(x) for x in [0.14,0.2,0.3,0.4, 0.5,0.6,0.7,0.8,0.9, 1,2,3,4, 5, 6,7,8,9, 10]]
xticklabel = [str(x) for x in [0.14, 0.5,1,2,5,10]]
xticklabels = [0.14,' ',' ',' ', 0.5,' ',' ',' ',' ', 1,' ',' ',' ', 5, ' ',' ',' ',' ', 10]
yticks = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8,9,10,20,30,40,50,60,70,80,90,100]
yticklabels = ['1','','','','','','','','',10,'','','','','','','','',100]
#xticklabels = [0.14,' ',' ',' ', 0.5,' ',' ',' ',' ', 1,' ',' ',' ', 5, ' ',' ',' ',' ', 10]
#xticklabels = [0.14,' ',' ',' ', 0.5,' ',' ',' ',' ', 1,' ',' ',' ', 5, ' ',' ',' ',' ', 10]
#ax.set_xticks([np.log10(x) for x in xticks])
#axis.set_xlim([np.log10(x) for x in [0.14,1]])
axis.set_xlim([np.log10(0.14),1])
axis.set_xticks(xticks)
axis.set_xticklabels(xticklabels,fontsize=13)
axis.set_yticks(yticks)
axis.set_yticklabels(yticklabels,fontsize=13)
axis.set_xlabel(r"$\bar{F}_{\rm 80\%}$ [Jy/beam]",fontsize=14)
axis.set_ylabel("N",fontsize=14)


#ax.set_xscale('log')
axis.set_yscale('log')


#ax.xaxis.set_major_formatter(FuncFormatter(lambda x,y: '{}'.format(math.log(x, 10))))
axis.legend(fontsize=12)
fig.tight_layout()
plt.show()

In [439]:
plt.savefig(output_dir+"Hist_Fmean_80.pdf")

In [297]:
print(xticklabel)
print(xticks)

['0.14', '0.5', '1', '2', '5']
[-0.8538719643217619, -0.3010299956639812, 0.0, 0.3010299956639812, 0.6989700043360189]


In [213]:
fig, ax = plt.subplots()
bins = np.logspace(0.001,1.7, 36)
ax.hist(whole_mean,bins=bins,edgecolor='black',color='grey')
ax.set_yscale('log')
ax.set_xscale('log')

In [19]:
output_dir = input_dir+"Ensemble/Paper/"

In [17]:
output_dir = input_dir+"PaperFigs/"

In [18]:
plt.savefig(output_dir+"Chi2_80vsFmean_2panel.pdf")

In [10]:
%matplotlib tk

In [19]:
output_dir

'/mnt/Secdrive/TSmain/TSmain/PaperFigs/'

In [200]:
output_dir

'/mnt/Secdrive/TSmain/TSmain/LCs_test/'

In [18]:
len(Sigs_80)

10109

In [43]:
plt.close()
fig, ax = plt.subplots()
bins = np.linspace(-10,10,41)
ax.hist(Sigs_80, bins=bins, color='grey',edgecolor='black',alpha=0.3)
ax.hist(Sigs_check, bins=bins, color='grey',edgecolor='black',alpha=1.0)
ax.plot(bins,2000*np.exp(-bins**2/2/1.0),ls='--',color='black')


#ax.plot(bins,2000*np.exp(-bins**2/2/1.1**2),ls='--',color="blue")
ax.plot([5,5],[0.1,4000],ls='-',color='red',alpha=0.7)
ax.plot([-5,-5],[0.1,4000],ls='-',color='red',alpha=0.7)
ax.set_xlabel(r"Stochasticity ($(F_{i} - \bar{F}_{\rm 80\%}) / \sigma_{\rm fid}$)",fontsize=13)
ax.set_ylabel("# of Measurments",fontsize=13)
ax.tick_params(labelsize=12)
ax.set_ylim([0.8,4000])
ax.set_xlim([-7.5,7.5])
ax.set_yscale('log')

# Stochasticity Histogram

In [58]:
plt.close()
fig, ax = plt.subplots()
bins = np.linspace(-10,10,41)
xplotter= np.linspace(-10,10,201)
xbins = bins[1:]-0.5*(bins[1]-bins[0])
Sig_hist = ax.hist(Sigs_80, bins=bins, color='grey',edgecolor='black',alpha=0.3)
ax.hist([Sigs_check,[5.1]], bins=bins, color=['grey','red'],edgecolor='black',alpha=1,stacked=True)
Gsol, Gsolerr = LCA.Gaussfit(xbins,Sig_hist[0])#,np.sqrt(Sig_hist[0]))

Stch_stdev = np.std(Sigs_check)
Stch_mean = np.mean(Sigs_check)
#print(np.std(frac_slope_novar))
#print(np.mean(frac_slope_novar))
print(Gsol, Gsolerr)
print('Stch_slope_stdev: '+str(Stch_stdev))
print('Stch_slope_mean: '+str(Stch_mean))

# Generate gaussian function with the mean and standard deviation of the stochasticities
ax.plot(xplotter,1916*np.exp(-(xplotter-Stch_mean)**2/2/Stch_stdev**2),ls='--',color='black')

#ax.plot(bins,2000*np.exp(-bins**2/2/1.0),ls='--',color='black')
#ax.plot(xplotter,Gsol[0]*np.exp(-xplotter**2/2/Gsol[2]**2),ls='--',color='black')
#ax.plot(bins,2000*np.exp(-bins**2/2/1.1**2),ls='--',color="blue")
ax.plot([5,5],[0.1,4000],ls='--',color='red',alpha=0.7)
ax.plot([-5,-5],[0.1,4000],ls='--',color='red',alpha=0.7)
ax.set_xlabel(r"Stochasticity ($(F_{i} - \bar{F}_{\rm 80\%}) / \sigma_{\rm fid}$)",fontsize=15)
ax.set_ylabel("# of Measurments",fontsize=15)
ax.tick_params(labelsize=14)
ax.set_ylim([0.8,4000])
ax.set_xlim([-7.5,7.5])
ax.set_yscale('log')
fig.tight_layout()
print(Gsol, Gsolerr)

[ 1.90993005e+03 -2.32459198e-02  1.02737018e+00] [1.14445889e+01 7.10854399e-03 7.10854289e-03]
Stch_slope_stdev: 1.0929122854233113
Stch_slope_mean: 0.010708715202725656
[ 1.90993005e+03 -2.32459198e-02  1.02737018e+00] [1.14445889e+01 7.10854399e-03 7.10854289e-03]


In [60]:
output_dir

'/mnt/Secdrive/TSmain/TSmain/LCs_test/'

In [25]:
max(Sig_hist[0])

1916.0

In [32]:
output_dir

'/mnt/Secdrive/TSmain/TSmain/LCs_test/'

In [92]:
print(np.array(Sigs)[np.where(np.array(Sigs)>5.0)])
np.array(Sigs)[np.where(np.array(Sigs)>5.0)]

[5.29969947 5.38549236 5.51814987 6.59087896 6.23932955]


In [59]:
plt.savefig(output_dir+"Hist_Sigs_overplot.pdf")

In [251]:

i = 6
fig, ax = plt.subplots()
ax.scatter(FAPs_Lin,FAPs_Sin_mod,color='grey',marker='o',s=20)
#ax.scatter(np.array(FAPs_Lin)[Sin_mod_d[0][:i]],np.array(FAPs_Sin_mod)[Sin_mod_d[0][:i]],edgecolors='black',facecolor='none',marker='o',s=50)
ax.scatter(np.array(FAPs_Lin)[Ps], np.array(FAPs_Sin_mod)[Ps],color="Red",marker='o',s=20,label = "Protostar")
ax.plot([1e-3,1e-3],[1e-15,10], color='blue',linestyle='--')
ax.plot([1e-15,10],[1e-3,1e-3], color='blue',linestyle='--')
ax.legend(loc='best')
ax.set_xlim([1e-15,10])
ax.set_ylim([1e-15,10])
ax.set_ylabel(r'FAP$_{Sine}$*N$_{freq}$')
ax.set_xlabel(r'FAP$_{Lin}$')
ax.set_xscale('log')
ax.set_yscale('log')

4.0

In [49]:
plt.savefig(output_dir+"FAPsvsFAPl.pdf")


In [50]:
output_dir

'/mnt/Secdrive/TSmain/InCanada/Ensemble/Paper/'